<a href="https://colab.research.google.com/github/65-1157/65-1157/blob/main/Dynamic-Programming%20-%202025_10_27_28_DP_Detalhes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programação Dinâmica (DP) em PLN

**Objetivo da DP:** Resolver problemas complexos dividindo-os em subproblemas sobrepostos e armazenando os resultados. Em PLN, otimiza o processamento de sequências.

**Aplicações Neste Notebook:**

* **Distância de Edição (Levenshtein):** Medição de similaridade entre strings. Essencial para correção ortográfica e busca aproximada.
* **Viterbi (HMM Simplificado):** Algoritmo fundamental para inferir a **sequência ótima de estados ocultos** (tags) a partir de observações (palavras).

**Glossário Rápido:**

* **HMM:** Hidden Markov Model (Modelo Oculto de Markov). Estrutura probabilística de sequências.
* **POS Tagging:** Part-of-Speech Tagging (Classificação Gramatical).

## 1) Distância de Edição (Levenshtein)

**Conceito:** A Distância de Levenshtein é o número mínimo de edições (inserções, deleções, substituições) necessárias para transformar uma string `A` em uma string `B`.

**Definição Recursiva (Programação Dinâmica):**
A solução é construída a partir da célula $\mathbf{dp}[i][j]$, que representa o custo de transformar o prefixo $\mathbf{A}_{1..i}$ em $\mathbf{B}_{1..j}$.

**Fórmula de Transição:**

| Operação | Origem da Tabela | Custo da Célula $\mathbf{dp}[i][j]$ |
| :--- | :--- | :--- |
| **Deleção de $A_{i}$** (veio de cima) | $\mathbf{dp}[i-1][j]$ | $\mathbf{dp}[i-1][j] + 1$ |
| **Inserção de $B_{j}$** (veio da esquerda) | $\mathbf{dp}[i][j-1]$ | $\mathbf{dp}[i][j-1] + 1$ |
| **Substituição/Match** (veio da diagonal) | $\mathbf{dp}[i-1][j-1]$ | $\mathbf{dp}[i-1][j-1] + \text{custo}(\mathbf{A}_{i}, \mathbf{B}_{j})$ |

**Mecanismo de Solução (Backtrace):**

* O valor final $\mathbf{dp}[n][m]$ é o custo mínimo.
* Para **reconstruir o caminho**, o algoritmo retrocede (`backtrace`), sempre escolhendo o vizinho que levou ao valor mínimo atual.
* **Complexidade:** $\mathcal{O}(n \cdot m)$ em tempo e memória.

In [ ]:
import pandas as pd
from typing import List, Tuple, Dict
import math

In [ ]:
def distancia_edicao(a: str, b: str) -> List[List[int]]:
    """
    Compute e retorna a tabela DP (Distância de Edição de Levenshtein) completa.
    dp[i][j] = custo mínimo para transformar a[:i] em b[:j].
    """
    n, m = len(a), len(b) # computar o tamanho das palavras.
    # palavra a = ficará na horizontal
    # palavra b = ficará na vertical
    # Inicializa a tabela (n+1 linhas, m+1 colunas)
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    print('matriz de inicio -- step 0')
    print(dp)
    breakpoint()

    # Casos base: Linha 0 (deletar) e Coluna 0 (inserir)
    for i in range(n + 1): # varredura na linha
        dp[i][0] = i
    for j in range(m + 1): # varredura na coluna
        dp[0][j] = j
    print('matriz de inicio -- step 1')
    print(dp)
    breakpoint()

    # Preenchimento da Tabela Programação Dinâmica
    for i in range(1, n + 1): # varredura na linha
        for j in range(1, m + 1): # varredura na coluna
            # Custo da substituição/match: 0 se caracteres iguais(match), 1 senão.
            custo_sub = 0 if a[i-1] == b[j-1] else 1

            dp[i][j] = min(
                dp[i-1][j] + 1,        # 1. Deleção de a[i-1] (veio de cima)
                dp[i][j-1] + 1,        # 2. Inserção de b[j-1] (veio da esquerda)
                dp[i-1][j-1] + custo_sub # 3. Substituição/Match (veio da diagonal)
            )
        print('matriz de inicio -- step 2')
        print(dp)
        #breakpoint()

    print('matriz de inicio -- step 3')
    print(dp)
    breakpoint()

    return dp # matriz de programacao dinamica

def print_dp_table(a: str, b: str, dp: List[List[int]]):
    # a = palavra na horizontal (input)
    # b = palavra na vertical (output)
    # dp = matriz de custo minimo (já apurado)
    """
    Show tabela DP formatada usando pandas para maior clareza.
    """
    # Cria os rótulos de linha e coluna
    linhas = ['ε'] + list(a) # ε = string vazia
    colunas = ['ε'] + list(b)

    # Cria o DataFrame
    df = pd.DataFrame(dp, index=linhas, columns=colunas)
    print(df)
    breakpoint()


def backtrace_operacoes(a: str, b: str, dp: List[List[int]]) -> List[Tuple]:
    """
    Reconstruct sequência de operações ótima a partir da tabela DP.
    Prioriza MATCH sobre SUB, e SUB sobre INSERT/DELETE,
    para maior clareza na reconstrução.
    """
    i, j = len(a), len(b)
    ops = []
    print('avaliacao_custo_step 4')
    print(f"i: {i}, j: {j}")
    # breakpoint()

    while i > 0 or j > 0: # indices de linhas e colunas
        # 1. Match/Substituição (diagonal): A escolha mais frequente
        custo_sub = 0 if (i > 0 and j > 0 and a[i-1] == b[j-1]) else 1
        print('avaliacao_custo_step 5')
        print(f"i: {i}, j: {j}, custo_sub: {custo_sub}")
        # breakpoint()

        if i > 0 and j > 0 and dp[i][j] == dp[i-1][j-1] + custo_sub:
            # É uma Substituição ou Match
            if custo_sub == 0:
                ops.append(("MATCH", a[i-1]))
            else:
                ops.append(("SUB", a[i-1], b[j-1]))
            i -= 1
            j -= 1

        # 2. Deleção (cima): Veio de dp[i-1][j] + 1
        elif i > 0 and dp[i][j] == dp[i-1][j] + 1:
            ops.append(("DELETE", a[i-1]))
            i -= 1

        # 3. Inserção (esquerda): Veio de dp[i][j-1] + 1
        elif j > 0 and dp[i][j] == dp[i][j-1] + 1:
            ops.append(("INSERT", b[j-1]))
            j -= 1

        else:
            # para caso esdrúxulo
            raise Exception("Erro no backtrace: não foi possível determinar a operação ótima.")
        print('avaliacao_custo_step 6')
        print(f"i: {i}, j: {j}")
        print(f"ops: {ops}")
        # breakpoint()

    return list(reversed(ops)) # operacoes em ordem reversa

In [ ]:
A, B = "gatos", "gados" # inputs para esse caso
print(f"Palavra A: '{A}', Palavra B: '{B}'")

Palavra A: 'gatos', Palavra B: 'gados'


In [ ]:
dp = distancia_edicao(A, B)
print("\nTabela DP:")
print_dp_table(A, B, dp)

matriz de inicio -- step 0
[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
> /tmp/ipython-input-2175260697.py(16)distancia_edicao()
     14 
     15     # Casos base: Linha 0 (deletar) e Coluna 0 (inserir)
---> 16     for i in range(n + 1): # varredura na linha
     17         dp[i][0] = i
     18     for j in range(m + 1): # varredura na coluna

ipdb> c
matriz de inicio -- step 1
[[0, 1, 2, 3, 4, 5], [1, 0, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0], [3, 0, 0, 0, 0, 0], [4, 0, 0, 0, 0, 0], [5, 0, 0, 0, 0, 0]]
> /tmp/ipython-input-2175260697.py(25)distancia_edicao()
     23 
     24     # Preenchimento da Tabela Programação Dinâmica
---> 25     for i in range(1, n + 1): # varredura na linha
     26         for j in range(1, m + 1): # varredura na coluna
     27             # Custo da substituição/match: 0 se caracteres iguais(match), 1 senão.

ipdb> c
matriz de inicio -- step 2
[[0, 1, 2, 3, 4, 5], [1, 0, 1, 2, 3, 4], [2, 0,

In [ ]:
print("\nOperações (Backtrace):")
for op in backtrace_operacoes(A, B, dp):
    print(op)
dist = dp[len(A)][len(B)]

print(f"\nDistância de Edição: {dist}")

# MELHORIA 1: Visualização da Tabela DP
print("\nTabela DP:")
print_dp_table(A, B, dp)

# MELHORIA 2: Backtrace mais limpo
print("\nOperações (Backtrace Limpo):")
for op in backtrace_operacoes(A, B, dp):
    print(op)


Operações (Backtrace):
avaliacao_custo_step 4
i: 5, j: 5
avaliacao_custo_step 5
i: 5, j: 5, custo_sub: 0
avaliacao_custo_step 6
i: 4, j: 4
ops: [('MATCH', 's')]
avaliacao_custo_step 5
i: 4, j: 4, custo_sub: 0
avaliacao_custo_step 6
i: 3, j: 3
ops: [('MATCH', 's'), ('MATCH', 'o')]
avaliacao_custo_step 5
i: 3, j: 3, custo_sub: 1
avaliacao_custo_step 6
i: 2, j: 2
ops: [('MATCH', 's'), ('MATCH', 'o'), ('SUB', 't', 'd')]
avaliacao_custo_step 5
i: 2, j: 2, custo_sub: 0
avaliacao_custo_step 6
i: 1, j: 1
ops: [('MATCH', 's'), ('MATCH', 'o'), ('SUB', 't', 'd'), ('MATCH', 'a')]
avaliacao_custo_step 5
i: 1, j: 1, custo_sub: 0
avaliacao_custo_step 6
i: 0, j: 0
ops: [('MATCH', 's'), ('MATCH', 'o'), ('SUB', 't', 'd'), ('MATCH', 'a'), ('MATCH', 'g')]
('MATCH', 'g')
('MATCH', 'a')
('SUB', 't', 'd')
('MATCH', 'o')
('MATCH', 's')

Distância de Edição: 1

Tabela DP:
   ε  g  a  d  o  s
ε  0  1  2  3  4  5
g  1  0  1  2  3  4
a  2  1  0  1  2  3
t  3  2  1  1  2  3
o  4  3  2  2  1  2
s  5  4  3  3  2  1